In [ ]:
import torch
x = torch.tensor([
    [ 0.0470, -0.0431,  0.0421],
    [ 0.0282, -0.0119,  0.0402],
    [0.0181, 0.0000, 0.0000],
    [-0.0017,  0.0001, -0.0004],
    [ 4.1447e-03, -2.7173e-03, -2.3842e-07]
    ])
x.softmax(1)

In [ ]:
torch.softmax(-x*20, 1)

In [ ]:
torch.softmax(x.exp()-1, 1)

In [ ]:
x.exp()-1

In [ ]:
x.exp()/2

In [ ]:
x1 = x - x.min(1, keepdim=True)[0] + 1e-3
x1 / x1.sum(1, keepdim=True)

# TOXIC dpo

- turn exach pair into ds row
- shuffle

In [ ]:
from datasets import load_dataset
dataset = load_dataset('unalignment/toxic-dpo-v0.1') # this should give a a bigger # use map in batched mode to return more rows than it got
def transform(row):
    return {
        "chosen": [{'role':'user', 'content': row['prompt']},{'role':'assistant', 'content': row['chosen']}],
        "rejected": [{'role':'user', 'content': row['prompt']},{'role':'assistant', 'content': row['rejected']}]
    }

dataset1 = dataset.map(transform)
dataset1

## Download model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "NousResearch/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# HelpSteer2

https://github.com/jondurbin/bagel/blob/3c7d2410a5a5ad2fd31b63529ef541135feefce4/bagel/data_sources/helpsteer.py#L4

In [ ]:
from datasets import load_dataset
dataset = load_dataset('nvidia/HelpSteer2')
dataset

In [ ]:
import re
from loguru import logger

REFUSALS = [
    re.compile(regexp)
    for regexp in [
        "(against|violate) my programming",
        "openai",
        "language model",
        "large language",
        "as an? (ai|generative language|gpt|bot)",
        "important to note",
        "i do(n't| not) (possess|have|exhibit) (personal|consciousness|subjective)",
        "personal (feelings|thoughts|emotions|desires|experiences|goals|objective|belief)",
        "(can('t| ?not)|w(on't|will not)|unable.?) (\\w+\\s)+(with (that|your)|your \\w+|provide)",
        "my limitations",
        "the limitations of my",
        "my abilities",
        "violates my",
        "i (can('t| ?not)|w(on't|will not)|am (not |un)able.?).{0,30}(you are|you're|your )",
        "please note that",
        "unethical|illegal|dangerous",
        "a text-based",
        "(engag(e|ing)|participat(e|ing)|be involved (in|with)|promot(e|ing)|discuss(ing)?|provid(e|ing))( in)?(\\s*\\w+ that)?(\\s+potentially)? (derogatory|inappropriate|offensive|discriminate|discriminatory|sexist|unacceptable|immoral|unethical|unacceptable|hateful|harmful)",
        "i am commited to",
        "adhere to safety guidelines",
        "maintain user safety",
        "about something else instead",
    ]
]

def has_refusal(response):
    """Check if a response has a refusal - not perfect, but good enough..."""
    for regexp in REFUSALS:
        if m := regexp.search(response, re.I):
            logger.warning(f"Refusal? {m.group()}")
            return True
    return False

.filter(
        lambda item: not has_refusal(item["response"])
    )

In [ ]:
def load_data(known_uids=set([]), **_):
    """HelpSteer dataset for DPO."""
    logger.info("Loading HelpSteer dataset...")
    dataset = load_dataset("nvidia/HelpSteer", split="train").filter(
        lambda item: not has_refusal(item["response"])
    )

    # Use the highest correctness items as the "chosen" responses.
    logger.info("Generating DPO pairs...")
    options = {}
    for item in dataset:
        uid = get_uid(item["prompt"])
        if uid not in options:
            options[uid] = []
        item["score"] = (
            item["helpfulness"]
            + item["correctness"]
            + item["coherence"]
            + item["complexity"]
            + item["verbosity"]
        )
        options[uid].append(item)

    # Select the chosen and rejected items.
    data = []
    for uid, responses in options.items():
        if len(responses) < 2:
            continue
        if uid in known_uids:
            continue
        known_uids.add(uid)
        chosen = None
        chosen_score = 0
        rejected = None
        rejected_score = 0
        for item in responses:
            if item["correctness"] == 4:
                if chosen:
                    if item["score"] > chosen["score"]:
                        rejected = chosen["response"]
                        rejected_score = chosen["score"]
                        chosen = item
                        chosen_score = item["score"]
                        break
                else:
                    chosen = item
                    chosen_score = item["score"]
            else:
                if not rejected:
                    rejected = item["response"]
                    rejected_score = item["score"]
        if chosen and rejected and chosen["response"] != rejected:
            logger.success(f"Found DPO pair: {chosen_score} vs {rejected_score}")
            data.append(
                {
                    "id": uid,
                    "source": "helpsteer",
                    "prompt": item["prompt"],
                    "chosen": chosen["response"],
                    "rejected": rejected,
                    "conversations": None,
                }
            )
    return Dataset.from_list(data)

In [ ]:
dataset['train']['prompt']

## Is norm mean same as mse?

In [13]:
import torch
import torch.nn.functional as F
import torch.nn as nn

x = torch.randn(2, 3)
y = torch.randn(2, 3)



(tensor(2.0638), tensor(3.5189), tensor(1.2046), tensor(7.2274))

In [14]:
F.mse_loss(x, y), torch.norm(y-x, p=2).nanmean(), F.l1_loss(x, y), torch.norm(y-x, p=1).nanmean()

(tensor(2.0638), tensor(3.5189), tensor(1.2046), tensor(7.2274))